In [25]:
#importing libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
import json

# Task 1: Data Collection via REST APIs

In [30]:
#Quering weather data from open-meteo API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.89193,
	"longitude": 12.51133,
	"start_date": "2025-07-08",
	"end_date": "2026-01-08",
	"hourly": ["temperature_2m", "wind_speed_10m", "precipitation", "relative_humidity_2m"],
}
responses = requests.get(url, params=params)
print(responses.status_code)
weather_data = responses.json()

#Saving the data to a file
with open('data/weather_data.json', 'w') as f:
    json.dump(weather_data, f, indent=4)

200


In [ ]:
#Quering weather data from open-meteo API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.89193,
	"longitude": 12.51133,
	"start_date": "2025-07-08",
	"end_date": "2026-01-08",
	"hourly": ["temperature_2m", "wind_speed_10m", "precipitation", "relative_humidity_2m"],
}
responses = requests.get(url, params=params)
print(responses.status_code)
weather_data = responses.json()

#Saving the data to a file
with open('weather_data', 'w') as f:
    json.dump(weather_data, f)

# Task 2: Data Cleaning and Integration

# Task 3: Exploratory Data Analysis

# Task 4: Machine Learning

# Task 5: Evaluation and Interpretation